In [ ]:
wb_cor=wb_t.corrwith(wb_t['age']).drop('age').sort_values()

In [ ]:
man=pd.read_table('../Blood-DNA-Methylation-Atlas/data/ref/EPIC.hg38.manifest.tsv',index_col='Probe_ID')[['CpG_chrm','CpG_beg']]
man=man.dropna().copy()
man['CpG_beg']=man['CpG_beg'].astype(int)
man.columns=[0,1]
man=man[man[0].isin('chr'+str(c) for c in range(1,23))].copy()
#man['cg']=man.index

In [ ]:
man=man.merge(wb_cor.rename('cor'),left_index=True,right_index=True)
man['pos']=man[0].str[3:].astype(int)*10**9+man[1]
man=man.sort_values('pos')
man.shape
#man['bin']=man['cor']>0

In [12]:
cor450=pd.read_csv('data/res/cor450.csv',index_col=0)
man=man.merge(cor450,left_index=True, right_index=True,how='left')

In [110]:
pl['s']=pl['chr'].str[3:].astype(int)*10**9+pl['b']
pl['t']=pl['chr'].str[3:].astype(int)*10**9+pl['e']

In [163]:
genc=pd.read_table('data/ref/gencode.txt',index_col=0)
genc.columns=range(1,6)
genc=genc[genc[1].isin(['chr'+str(c) for c in range(1,23)])].copy()
genc['tss']=genc.apply(lambda x: int(x[1][3:])*10**9+x[3] if x[2]=='+' else int(x[1][3:])*10**9+x[4],axis=1)
genc['tts']=genc.apply(lambda x: int(x[1][3:])*10**9+x[4] if x[2]=='+' else int(x[1][3:])*10**9+x[3],axis=1)
genc=genc[genc[5].str[:4]!='ENSG'].copy()
genc=genc[genc[5].str[:4]!='LINC'].copy()
genc=genc[genc[5].str[:3]!='MIR'].copy()
genc=genc[genc[5].str[:3]!='RPL'].copy()
genc=genc[genc[5].str[-3:]!='-DT'].copy()
genc=genc[genc[5].str[-4:-1]!='-AS'].copy()
ts={}
ts['s']=genc[[5,'tss']].drop_duplicates().groupby('tss')[5].apply(min)
ts['t']=genc[[5,'tts']].drop_duplicates().groupby('tts')[5].apply(min)

In [164]:
%%time
dis={}
gen={}
for typ in ['s','t']:
    dis[typ]=[]
    gen[typ]=[]
    for i,b in pl[['s','t']].iterrows():
        fns=fn(ts[typ].index,b['s'])
        fnt=fn(ts[typ].index,b['t'])    
        if b['s']<fns<b['t']:
            dis[typ].append(0)
            gen[typ].append(ts[typ].loc[fns])        
        elif b['s']<fnt<b['t']:
            dis[typ].append(0)   
            gen[typ].append(ts[typ].loc[fnt])        
        elif np.abs(b['s']-fns)<=np.abs(b['t']-fnt):
            dis[typ].append(np.abs(b['s']-fns))
            gen[typ].append(ts[typ].loc[fns])
        else:
            dis[typ].append(np.abs(b['t']-fnt))
            gen[typ].append(ts[typ].loc[fnt])

CPU times: user 477 ms, sys: 24.1 ms, total: 501 ms
Wall time: 470 ms


In [165]:
pl['dis_s']=dis['s']
pl['gen_s']=gen['s']
pl['dis_t']=dis['t']
pl['gen_t']=gen['t']

In [ ]:
pl['Gene']=pl.apply(lambda x: gen(x),axis=1)

In [184]:
plpr=pl.drop(['s','t'],axis=1).copy()

In [185]:
plpr.columns=['Number of CpGs','Chr','DMR Start','DMR End','Corr (Epic)','Corr (450k)', 
            'HMR','Gain or Loss','Distance to TSS','Nearest TSS','Distance to TTS','Nearest TTS','Gene']
plpr[['Gene','Nearest TSS','Distance to TSS','Nearest TTS','Distance to TTS','Number of CpGs','Chr','DMR Start','DMR End','Corr (Epic)','Corr (450k)', 
            'HMR','Gain or Loss']].round(2).to_csv('data/res/genes.csv')